In [80]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp

import VariationalBayes as vb
from copy import deepcopy

import scipy as osp
from scipy import stats


In [104]:
class Kernel(object):
    def __init__(self, c=1.0, beta=-0.5):
        self.c = c
        self.beta = beta
        
    def k(self, x, y):
        xy_diff = x - y
        return (c + np.dot(xy_diff, xy_diff)) ** beta

    
class MVNModel(object):
    def __init__(self, obs, obs_info_mat, prior_loc):
        self.prior_loc = deepcopy(prior_loc)
        self.obs_dim = len(obs)
        self.obs = obs
        self.obs_info_mat = deepcopy(obs_info_mat)
        self.prior_info_mat = np.eye(self.obs_dim)

        self.theta = np.zeros(self.obs_dim)
        
    def get_log_post(self, broadcast=False):
        if broadcast:
            obs_centered = np.expand_dims(self.obs, 1) - self.theta
            theta_centered = self.theta - np.expand_dims(self.prior_loc, 1)
        else:
            obs_centered = self.obs - self.theta
            theta_centered = self.theta - self.prior_loc
            
        log_lik =  -0.5 * np.einsum(
            'i...,ij,j...->...',
            obs_centered, self.obs_info_mat, obs_centered)

        log_prior =  -0.5 * np.einsum(
            'i...,ij,j...->...',
            theta_centered, self.prior_info_mat, theta_centered)
        
        return log_lik + log_prior
    
    def get_post_cov(self):
        return np.linalg.inv(self.obs_info_mat + self.prior_info_mat)

    def get_post_mean(self):
        post_cov = self.get_post_cov()
        post_suff_stat = np.matmul(self.obs_info_mat, obs) + \
                         np.matmul(self.prior_info_mat, self.prior_loc)
        return np.matmul(post_cov, post_suff_stat)
    
    def set_theta_get_log_post(self, theta):
        self.theta = theta
        return self.get_log_post()



In [105]:
obs_dim = 2
obs = np.array([0.2, 0.5])
obs_info_mat = 30 * np.eye(obs_dim)
prior_loc = np.zeros(obs_dim)
theta = np.full(obs_dim, 0.2)

mvn_model = MVNModel(obs, obs_info_mat, prior_loc)
print(mvn_model.get_post_mean())
print(obs)



[ 0.19354839  0.48387097]
[ 0.2  0.5]


In [106]:
num_draws = 10
theta_draws = np.transpose(osp.stats.multivariate_normal.rvs(
    size=num_draws, mean=mvn_model.get_post_mean(), cov=mvn_model.get_post_cov()))


In [109]:
get_log_post_grad = autograd.jacobian(mvn_model.set_theta_get_log_post)
get_log_post_grad(theta_draws[:, 0])

post_grads = np.array([ get_log_post_grad(theta) for theta in np.transpose(theta_draws) ])
print(post_grads)


[[ -4.61121698 -12.19094068]
 [ -4.67825364  -7.71340681]
 [  3.8427866    5.31848418]
 [  3.24158064  -0.11408701]
 [  2.97613179   6.85717704]
 [  2.67529261  -2.51249715]
 [ -1.00787815   0.1594007 ]
 [ -1.14406884  -1.34877388]
 [ 12.76331752 -13.6554587 ]
 [  1.85667867  -2.35800555]]
